## Temat 1. Własna sieć neuronowa
Celem projektu jest samodzielne zaimplementowanie prostej sieci neuronowej i przetestowanie jej na wybranym przez siebie przykładzie.

Podstawowym założeniem projektu jest własnoręczne zaimplementowanie automatycznego różniczkowania w celu wyznaczania pochodnych warstw potrzebnych do wstecznej   propagacji błędów. Dopuszczalne jest wykorzystanie różniczkowanie w przód (ang. Forward Accumulation), różniczkowanie w tył (ang. Reverse Accumulation) oraz różniczkowanie oparte o generację kodu (ang. source-to-source differentiation).

Do implementacji optymalizatorów dokonujących właściwego uczenia sieci można wykorzystać kod zamieszczony w książce "Algorithms for Optimization" [1].

klasyfikacja cyfr (Digits MNIST dataset)  
Bibliografia:  
[1] Mykel J. Kochenderfer, Tim A. Wheeler, 2019, Algorithms for Optimization, MIT Press.  
[2] Martin T. Hagan et. al, Neural Network Design, pp. 915-918, url: https://hagan.okstate.edu/NNDesign.pdf  
[3] 3blue1brown: czym są sieci neuronowe? https://www.youtube.com/watch?v=aircAruvnKk  
  


In [108]:
using Pkg
Pkg.activate("./project")

  Activating project at `e:\Documents\1. Studia\MAGISTERSKIE\Algorytmy w inż danych\Custom-Julia-NN\project`


In [282]:
Pkg.add("MLDatasets")
Pkg.add("IJulia")
Pkg.add("StableRNGs")


   Resolving package versions...
  No Changes to `E:\Documents\1. Studia\MAGISTERSKIE\Algorytmy w inż danych\Custom-Julia-NN\project\Project.toml`
  No Changes to `E:\Documents\1. Studia\MAGISTERSKIE\Algorytmy w inż danych\Custom-Julia-NN\project\Manifest.toml`
   Resolving package versions...
  No Changes to `E:\Documents\1. Studia\MAGISTERSKIE\Algorytmy w inż danych\Custom-Julia-NN\project\Project.toml`
  No Changes to `E:\Documents\1. Studia\MAGISTERSKIE\Algorytmy w inż danych\Custom-Julia-NN\project\Manifest.toml`
   Resolving package versions...
  No Changes to `E:\Documents\1. Studia\MAGISTERSKIE\Algorytmy w inż danych\Custom-Julia-NN\project\Project.toml`
  No Changes to `E:\Documents\1. Studia\MAGISTERSKIE\Algorytmy w inż danych\Custom-Julia-NN\project\Manifest.toml`


# Load Dataset

In [283]:
using MLDatasets
using StableRNGs
train_x, train_y = MNIST.traindata(Float32);
test_x, test_y = MNIST.testdata(Float32);
train_size = size(train_x)
test_size = size(test_x)
train_x = reshape(train_x, (train_size[1],train_size[2],1,train_size[3]))
test_x = reshape(test_x, (test_size[1],test_size[2],1,test_size[3]))
println("Train X shape: ",size(train_x));
println("Test X shape: ",size((test_x)));
test_test_y = test_y
test_y= Y_labels  .== permutedims(test_y)
train_y= Y_labels .== permutedims(train_y)
println("Train Y shape: ",size(train_y));
println("Test Y shape: ",size((test_y)));

Train X shape: (28, 28, 1, 60000)
Test X shape: (28, 28, 1, 10000)
Train Y shape: (10, 60000)
Test Y shape: (10, 10000)


## Activation functions

In [284]:
function sigmoid(Z)
    """
    Sigmoid activation function
    """
    A = 1 ./ (1 .+ exp.(.-Z))
    return (A = A, Z = Z)
end

function relu(Z)
    """
        ReLU activation function
    """
    A = max.(0, Z)
    return (A=A, Z=Z)
end

relu (generic function with 1 method)

In [308]:
function initialise_model_weights(layer_dims, seed)
    """
        Funtion to initialise the parameters or weights of the network.
    """
    params = Dict()
    println(2:length(layer_dims))

    for l=2:length(layer_dims)
        printlnt(rand(StableRNG(seed), layer_dims[l], layer_dims[l-1]))
        params[string("W_", (l-1))] = rand(StableRNG(seed), layer_dims[l], layer_dims[l-1]) * sqrt(1 / layer_dims[l-1])
        params[string("b_", (l-1))] = zeros(layer_dims[l], 1)
    end

    return params
end

initialise_model_weights (generic function with 1 method)

In [310]:
xd = initialise_model_weights((1000,32,16,4,10),1)
xd

2:5


Dict{Any, Any} with 8 entries:
  "b_1" => [0.0; 0.0; … ; 0.0; 0.0;;]
  "W_3" => [0.146299 0.163273 … 0.0890796 0.0478303; 0.0193345 0.0591598 … 0.02…
  "W_2" => [0.103449 0.109158 … 0.0352753 0.164116; 0.0136715 0.0851234 … 0.156…
  "b_2" => [0.0; 0.0; … ; 0.0; 0.0;;]
  "b_3" => [0.0; 0.0; 0.0; 0.0;;]
  "b_4" => [0.0; 0.0; … ; 0.0; 0.0;;]
  "W_1" => [0.0185055 0.00504804 … 0.00063453 0.00564094; 0.00244564 0.00160207…
  "W_4" => [0.292597 0.47187 0.429261 0.202521; 0.038669 0.222835 0.349357 0.16…

      Status `E:\Documents\1. Studia\MAGISTERSKIE\Algorytmy w inż danych\Custom-Julia-NN\project\Project.toml`
  [7073ff75] IJulia v1.23.2
  [eb30cadb] MLDatasets v0.5.15
  [860ef19b] StableRNGs v1.0.0
